In [1]:
import dspy
import pandas

/home/sylvia/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_data = pandas.read_excel("300 revised.xlsx")
df_data.columns

Index(['discussion_post_id', 'discussion_post_content_clean',
       'discussion_post_classification', 'discussion_topic', 'course_title',
       'course_desc', 'context_independence', 'parent_discussion_post_id',
       'llama_70b_ta_response',
       'Clarify misunderstandings and address questions',
       'Deepen Disciplinary Understanding', 'Develop Higher-Order Thinking',
       'Enhance Metacognitive Awareness',
       'Foster Collaborative Knowledge Construction and Social Presence'],
      dtype='object')

In [3]:
df_topic_ann = pandas.read_csv("300_with_context_annotation.csv")
df_topic_ann.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'discussion_post_id',
       'discussion_post_content_clean', 'discussion_post_classification',
       'discussion_topic', 'course_title', 'course_desc',
       'context_independence', 'parent_discussion_post_id', 'mellon_id',
       'discussion_context', 'llama_70b_ta_response_with_similar',
       'Clarify misunderstandings and address questions',
       'Deepen Disciplinary Understanding', 'Develop Higher-Order Thinking',
       'Enhance Metacognitive Awareness',
       'Foster Collaborative Knowledge Construction and Social Presence',
       'Unnamed: 18', 'Unnamed: 19'],
      dtype='object')

In [4]:
import random
random.seed(42)

# first set of selections
import numpy as np
np.random.seed(42)

selector_1 = np.random.choice(300, 150, replace=False)
selector_2 = np.random.choice(300, 150, replace=False)

In [5]:
selector_1

array([203, 266, 152,   9, 233, 226, 196, 109,   5, 175, 237,  57, 218,
        45, 182, 221, 289, 211, 148, 165,  78, 113, 249, 250, 104,  42,
       281, 295, 157, 238,  17, 164,  33,  24, 215, 119,   7,  90,  46,
        73,  93,  76, 286,  60,  77,  63, 234, 229, 111, 231, 180, 144,
       239,  75, 297, 278,  97,  92, 192,  25, 232,  59,   6, 185, 173,
        30,  22, 256,  56, 186, 108, 126, 230, 193, 129, 282,  82,  84,
        66, 288, 163, 154,  19, 124,  79, 114, 118,  72,  15,  10, 194,
       101,  68, 224,  37,  16, 179, 147, 274,  67, 228,  69,  31, 183,
       265, 225, 140,  18, 181,  96, 132, 262,  86, 248, 245, 116, 146,
       292, 197, 206,  55, 172, 184, 167, 139, 253,  38, 125, 195, 283,
       137, 112, 168, 117, 277, 271, 155, 176, 178,   2, 115, 143, 177,
       120, 210, 260, 127,  74,  29,  83])

In [ ]:
LEVEL_1 = open("rubrics/level_1.txt").read()
LEVEL_2 = open("rubrics/level_2.txt").read()
LEVEL_3 = open("rubrics/level_3.txt").read()
LEVEL_4 = open("rubrics/level_4.txt").read()
LEVEL_5 = open("rubrics/level_5.txt").read()


rubric_dict = {}

for col, desc in [
        ("Clarify misunderstandings and address questions", LEVEL_1),
        ("Deepen Disciplinary Understanding", LEVEL_2),
        ("Develop Higher-Order Thinking", LEVEL_3),
        ("Enhance Metacognitive Awareness", LEVEL_4),
        ("Foster Collaborative Knowledge Construction and Social Presence", LEVEL_5)
    ]:
    if col not in rubric_dict:
        rubric_dict.update({col: {}})
    line_descs = desc.split("\n")
    for l in line_descs:
        if l.startswith("Strong"):
            rubric_dict[col].update(
                {2: l.split(":")[1].strip()}
            )
        elif l.startswith("Weak"):
            rubric_dict[col].update(
                {1: l.split(":")[1].strip()}
            )
        elif l.startswith("Not Present"):
            rubric_dict[col].update(
                {0: l.split(":")[1].strip()}
            )
        elif l.startswith("Not Applicable"):
            rubric_dict[col].update(
                {"NA": l.split(":")[1].strip()}
            )

print(rubric_dict)


all_examples = {
    "Clarify misunderstandings and address questions": {
        0: [], 1: [], 2: [], "NA": []
    },
    "Deepen Disciplinary Understanding": {
        0: [], 1: [], 2: [], "NA": []
    },
    "Develop Higher-Order Thinking": {
        0: [], 1: [], 2: [], "NA": []
    },
    "Enhance Metacognitive Awareness": {
        0: [], 1: [], 2: [], "NA": []
    },
    "Foster Collaborative Knowledge Construction and Social Presence": {
        0: [], 1: [], 2: [], "NA": []
    },
}


counter = 0

for i, row in df_data.iterrows():
    post = row["discussion_post_content_clean"]
    resp = row["llama_70b_ta_response"]
    for col, desc in [
        ("Clarify misunderstandings and address questions", LEVEL_1),
        ("Deepen Disciplinary Understanding", LEVEL_2),
        ("Develop Higher-Order Thinking", LEVEL_3),
        ("Enhance Metacognitive Awareness", LEVEL_4),
        ("Foster Collaborative Knowledge Construction and Social Presence", LEVEL_5)
    ]:
        if pandas.notna(row[col]):
            if pandas.notna(row["course_desc"]):
                course_info = row["course_title"] + "\n\n" + row["course_desc"]
            else:
                course_info = row["course_title"]
            rating = int(row[col])
            if rating == 3:
                rating = "NA"
            if counter not in selector_1:
                all_examples[col][rating].append("DISCUSSION FORUM POST: " + post + "\nTA RESPONSE: " + resp)
    counter = counter + 1

counter = 0
for i, row in df_topic_ann.iterrows():
    post = row["discussion_post_content_clean"]
    resp = row["llama_70b_ta_response_with_similar"]
    for col, desc in [
        ("Clarify misunderstandings and address questions", LEVEL_1),
        ("Deepen Disciplinary Understanding", LEVEL_2),
        ("Develop Higher-Order Thinking", LEVEL_3),
        ("Enhance Metacognitive Awareness", LEVEL_4),
        ("Foster Collaborative Knowledge Construction and Social Presence", LEVEL_5)
    ]:
        if pandas.notna(row[col]):
            if pandas.notna(row["course_desc"]):
                course_info = row["course_title"] + "\n\n" + row["course_desc"]
            else:
                course_info = row["course_title"]
            rating = int(row[col])
            if rating == 3:
                rating = "NA"
            if counter not in selector_2:
                all_examples[col][rating].append("DISCUSSION FORUM POST: " + post + "\nTA RESPONSE: " + resp)
    counter = counter + 1


In [ ]:
import dotenv
dotenv.load_dotenv(".env")

lm = dspy.LM("openai/gpt-4.1-nano", cache=False, max_tokens=4000, temperature=0.8)
dspy.configure(lm=lm)

In [45]:
len(all_examples["Deepen Disciplinary Understanding"][0]) + len(all_examples["Deepen Disciplinary Understanding"][1]) + len(all_examples["Deepen Disciplinary Understanding"][2]) + len(all_examples["Deepen Disciplinary Understanding"]["NA"]) 

298

In [46]:
import copy
all_synthetic_examples = copy.deepcopy(all_examples)
for col in all_examples:
    for rating in all_examples[col]:
        og_len = len(all_examples[col][rating])
        all_synthetic_examples[col][rating] = []
        assert len(all_examples[col][rating]) == og_len
        assert len(all_synthetic_examples[col][rating]) == 0

In [47]:
from typing import List

class DataSynthesizer(dspy.Signature):
    """Given some student discussion forum posts and the corresponding teaching assistant responses, generate three similar post-response pairs. Make sure to follow the requirements for the teaching assistant response."""
    example_post_response_pairs: List[str] = dspy.InputField(desc="Pairs of discussion forum posts and the corresponding responses.")
    requirement = dspy.InputField(desc="The requirement for the synthesized teaching assistant responses.")
    new_post_response_pairs: List[str] = dspy.OutputField(desc="Synthetic examples of post-response pairs. Include the prefixes 'DISCUSSION FORUM POST:' and 'TA RESPONSE:'. Follow the requirement for the teaching assistant response in your generations.")

class Rewriter(dspy.Signature):
    """Given a pair of student discussion forum post and the corresponding teaching assistant response, rewrite the teaching assistant response to fit the specified requirement."""
    student_post = dspy.InputField()
    teaching_assistant_response = dspy.InputField()
    requirement = dspy.InputField()
    rewritten_response = dspy.OutputField(desc="Directly output the rewritten teaching assistant response. Make sure it follows the specific requirement.")

class DiscussionPostSynthesizer(dspy.Module):
    def __init__(self, callbacks=None):
        super().__init__(callbacks)
        self.synthesizer = dspy.ChainOfThought(DataSynthesizer)
        self.rewriter = dspy.ChainOfThought(Rewriter)
    
    def forward(self, example_post_response_pairs, requirement):
        all_new_examples = self.synthesizer(example_post_response_pairs=example_post_response_pairs,
                                            requirement=requirement).new_post_response_pairs
        fin_examples = []
        for ex in all_new_examples:
            if not ex.startswith("DISCUSSION FORUM POST:"):
                continue
            if not "TA RESPONSE:" in ex:
                continue
            fin_examples.append(ex)
            # post_resp = ex.split("TA RESPONSE:")
            # student_post = post_resp[0].replace("DISCUSSION FORUM POST:", "").strip()
            # ta_response = post_resp[1].strip()
            # new_rewritten = self.rewriter(student_post=student_post, teaching_assistant_response=ta_response, requirement=requirement).rewritten_response
            # fin_examples.append(
            #     "DISCUSSION FORUM POST: " + student_post + "\nTA RESPONSE: " + new_rewritten
            # )
        return fin_examples




In [ ]:
PER_CLASS_NUM = 300

import random
random.seed(42)

synthesizer = DiscussionPostSynthesizer()

for col in all_synthetic_examples:
    for rating in all_synthetic_examples[col]:
        # need to have a certain number of synthetic examples
        num_synthetic = PER_CLASS_NUM - len(all_examples[col][rating])
        pool = all_examples[col][rating]
        if len(pool) < num_synthetic:
            pool = all_examples[col][0] + all_examples[col][1] + all_examples[col][2] + all_examples[col]["NA"]
        while len(all_synthetic_examples[col][rating]) < num_synthetic:
            random_sample = random.choices(pool, k=5)
            rubric = rubric_dict[col][rating]
            all_synthetic_examples[col][rating].extend(synthesizer(example_post_response_pairs=random_sample, requirement=rubric))
            # print(len(all_synthetic_examples[col][rating]), "====")
            

In [40]:
all_synthetic_examples["Clarify misunderstandings and address questions"][0]

['DISCUSSION FORUM POST: I am trying to understand the difference between fixed and variable costs in business. Can someone explain the concept?\nTA RESPONSE: In business, fixed costs are expenses that do not change regardless of the level of production or sales, such as rent or salaries. Variable costs, on the other hand, fluctuate with the level of production, such as materials and labor costs that increase with higher output.',
 'DISCUSSION FORUM POST: Can you explain the importance of the scientific method in research?\nTA RESPONSE: The scientific method is a systematic approach used in research to formulate, test, and refine hypotheses. It involves several steps: observation, questioning, forming a hypothesis, conducting experiments, analyzing data, and drawing conclusions. This method ensures that research is conducted in an objective and reproducible manner.',
 "DISCUSSION FORUM POST: What are some common themes in Shakespeare's plays?\nTA RESPONSE: Shakespeare's plays feature s

In [49]:
import json

json.dump(all_synthetic_examples, open("all_synth_examples.json", "w+"))
json.dump(all_examples, open("all_examples.json", "w+"))